<a href="https://colab.research.google.com/github/victory2588/Machine-Learning/blob/main/%ED%8E%B8%EA%B3%A1%ED%95%98%EB%8A%94_%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# "작은 별" 악보를 ABC 표기로 표현
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2"
# "봄 나들이","나비야" 추가해서 돌리기
import numpy as np

# 계이름과 숫자를 상호 변환하는 표(딕셔너리)
note2num={'c':1,'d':2,'e':3,'f':4,'g':5,'a':6,'b':7}
num2note={1:'c',2:'d',3:'e',4:'f',5:'g',6:'a',7:'b'}

# ABC 표기를 시계열 데이터로 변환
def abc2timeseries(s):
  notes=s.split(' ')[2:]
  seq=[]
  for i in notes:
    seq.append([note2num[i[0]],int(i[1])])
  return seq

# 시계열 데이터를 ABC 표기로 변환
def timeseries2abc(t):
  s='tinynotation: 4/4'
  for i in t:
    s=s+' '+num2note[i[0]]+str(i[1])
  return s

# 원핫 코드로 변환하는 표
onehot=[[1,2],[2,2],[3,2],[4,2],[5,2],[6,2],[7,2],[1,4],[2,4],[3,4],[4,4],[5,4],[6,4],[7,4],[1,8],[2,8],[3,8],[4,8],[5,8],[6,8],[7,8]]

# 레이블을 원핫 코드로 변환
def to_onehot(l):
  t=[]
  for i in range(len(l)):
    a=np.zeros(len(onehot))
    a[onehot.index(list(l[i]))]=1.0
    t.append(a)
  return np.array(t)

# 시계열 데이터를 훈련 집합으로 자름
def seq2dataset(seq,window,horizon):
  X=[]; Y=[]
  for i in range(len(seq)-(window+horizon)+1):
    x=seq[i:(i+window)]
    y=(seq[i+window+horizon-1])
    X.append(x); Y.append(y)
  return np.array(X),np.array(Y)



In [ ]:
w=8 # 윈도우 크기
h=1 # 수평선 계수

seq=abc2timeseries(little_star)
X,Y=seq2dataset(seq,w,h)
print(X.shape,Y.shape)
print(X[0],Y[0])

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import tensorflow as tf

# 훈련 집합 구축
split=int(len(X)*1.0)   # 100%를 훈련집합으로 사용
x_train=X[0:split]; y_train=Y[0:split]
y_train=to_onehot(y_train)

# LSTM 모델 설계와 학습
model=Sequential()
model.add(LSTM(units=128,activation='relu',input_shape=x_train[0].shape))
model.add(Dense(y_train.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=300,batch_size=1,verbose=2)

# 학습된 모델로 편곡을 하는 함수
def arranging_music(model,first_measure,duration):
  music=first_measure
  for i in range(duration):
    p=model.predict(np.float32(np.expand_dims(music[-w:],axis=0)))
    music=np.append(music,[onehot[np.argmax(p)]],axis=0)
  return timeseries2abc(music)

new_song=arranging_music(model,x_train[0],50)

print(new_song)

(34, 8, 2) (34, 2)
[[1 4]
 [1 4]
 [5 4]
 [5 4]
 [6 4]
 [6 4]
 [5 2]
 [4 4]] [4 4]
Epoch 1/300
34/34 - 1s - loss: 2.7735 - accuracy: 0.0882 - 1s/epoch - 39ms/step
Epoch 2/300
34/34 - 0s - loss: 2.2355 - accuracy: 0.1471 - 181ms/epoch - 5ms/step
Epoch 3/300
34/34 - 0s - loss: 2.0574 - accuracy: 0.2353 - 197ms/epoch - 6ms/step
Epoch 4/300
34/34 - 0s - loss: 2.0438 - accuracy: 0.1471 - 176ms/epoch - 5ms/step
Epoch 5/300
34/34 - 0s - loss: 1.9065 - accuracy: 0.3529 - 175ms/epoch - 5ms/step
Epoch 6/300
34/34 - 0s - loss: 1.8599 - accuracy: 0.3529 - 181ms/epoch - 5ms/step
Epoch 7/300
34/34 - 0s - loss: 1.6795 - accuracy: 0.3235 - 170ms/epoch - 5ms/step
Epoch 8/300
34/34 - 0s - loss: 1.6031 - accuracy: 0.4706 - 182ms/epoch - 5ms/step
Epoch 9/300
34/34 - 0s - loss: 1.4034 - accuracy: 0.4412 - 182ms/epoch - 5ms/step
Epoch 10/300
34/34 - 0s - loss: 1.4044 - accuracy: 0.5000 - 172ms/epoch - 5ms/step
Epoch 11/300
34/34 - 0s - loss: 1.2364 - accuracy: 0.5294 - 182ms/epoch - 5ms/step
Epoch 12/300
34/

In [ ]:
import music21
music21.converter.parse(new_song).show('mid')